### LCEL 인터페이스
사용자 정의 체인을 가능한 쉽게 만들 수 있도록, 우리는 "Runnable" 프로토콜을 구현했습니다
표준 인터페이스에는 다음이 포함
* stream: 응답의 청크를 스트리밍합니다
* invoke: 입력에 대해 체인을 호출합니다
* batch: 입력 목록에 대해 체인을 호출합니다

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 대화 체인을 구성

model = ChatOpenAI(
    model="gpt-4o",
    temperature=0.0,
    max_tokens=2048,
)
prompt = ChatPromptTemplate.from_template("{topic} 으로 3행시 지어줘")
chain = prompt | model

In [3]:
# 입력 스키마
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
import json

# 스키마를 출력하는 함수
def print_schema(schema):
    print(json.dumps(schema, indent=4))

# 입력 스키마 정의
input_schema = {
    "type": "object",
    "properties": {
        "name": { "type": "string" },
        "age": { "type": "integer" , "minimum": 0 },
    },
    "required": ["name", "age"],
}

print_schema(input_schema)

model.input_schema.schema()

{
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer",
            "minimum": 0
        }
    },
    "required": [
        "name",
        "age"
    ]
}


{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

In [6]:
# 출력 스키마
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

In [7]:
# stream 실시간 출력
for s in chain.stream({"topic": "짜장면"}):
    print(s.content, end="", flush=True)

물론이죠! 여기 짜장면으로 지은 3행시입니다:

짜: 짜릿한 맛의 유혹에
장: 장난처럼 다가와도
면: 면발 한 가닥에 마음을 빼앗기네

In [8]:
# batch 단위 실행 AIMessage 응답
chain.batch(
    [
        {"topic": "배터리"},
        {"topic": "인스타"},
        {"topic": "파이썬"},
        {"topic": "아이유"},
        {"topic": "침착맨"},
    ],
    config={"max_concurrency": 3},
)

[AIMessage(content='배: 배터리가 없으면\n터: 터무니없이 불편해지지\n리: 리모컨도 못 쓰니까!', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 20, 'total_tokens': 50}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-3b848300-fccc-4ecd-9bc2-b7ff29cf2d1f-0'),
 AIMessage(content='물론이죠! "인스타"로 3행시를 지어볼게요.\n\n인: 인생의 순간들을\n스: 스쳐 지나가지 않게\n타: 타임라인에 담아봐요, 인스타그램!', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 19, 'total_tokens': 72}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ce6ed18-76e0-4944-9d2a-0313ff2da4fc-0'),
 AIMessage(content='물론이죠! "파이썬"으로 3행시를 지어볼게요.\n\n파: 파이썬은 정말 멋진 언어야,\n이: 이젠 누구나 쉽게 배울 수 있지.\n썬: 썬샤인처럼 밝게 빛나는 코드로!', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 21, 'total_tokens': 89}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs':

In [10]:
# Parallel: 병렬성
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야?") | model
chain2 = ChatPromptTemplate.from_template("{country} 의 면적은 얼마야?") | model

combined = RunnableParallel(capital=chain1, area=chain2)

combined.invoke({"country": "한국"})


{'capital': AIMessage(content='한국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로서 많은 중요한 정부 기관과 기업들이 위치해 있습니다.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_729ea513f7', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce66a5c2-8f4e-48fb-ac52-1b888c38e215-0'),
 'area': AIMessage(content='대한민국의 면적은 약 100,210 평방 킬로미터(km²)입니다. 이는 남한의 면적을 기준으로 한 수치입니다. 북한을 포함한 한반도 전체의 면적은 약 220,000 평방 킬로미터(km²)입니다.', response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 15, 'total_tokens': 83}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_927397958d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7702001e-a386-4443-a655-473d953d7205-0')}